In [1]:
# include useful folders
import sys
sys.path

['/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/IPython/extensions',
 '/root/.ipython']

In [2]:
# ! pip install dlblocks

In [3]:
sys.path.append("../vendors/mtl_girnet/data_prep/")

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [5]:
import json
import h5py
import numpy as np
import glob
import os
import random

In [6]:
# from twtokenize import tokenize
# from keras.utils import to_categorical
# from xml.dom import minidom

In [7]:
from tokensize_deepmoji import tokenize
from xml.dom import minidom

In [8]:
sents = {"N":-1 , "P" :1 , "NONE":0}

In [9]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_train.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_train = data

In [10]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_test.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_test = data

In [11]:
en_es_wssa_data = list(en_es_wssa_data_train) + list(en_es_wssa_data_test)

In [12]:
xmldoc = minidom.parse("../vendors/mtl_girnet/data_prep/data_cm_senti/general-tweets-train-tagged.xml")
tweets = xmldoc.getElementsByTagName('tweet')

sents = {"N":-1 , "P" :1 , "NEU":0 , 'NONE':0 , "P+" : 1 , "N+":-1 }


es_tass1_data = []

for i in range( len(tweets)-1) :
    if i == 6055:
        continue # bad jogar
    textt = tweets[i].getElementsByTagName('content')[0].childNodes[0].data
    words = tokenize( textt )
    sentiment = tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('value')[0].childNodes[0].data
    assert len(tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('entity'))==0
    es_tass1_data.append({'text':textt , 'tokens':words , 'sentiment': sents[sentiment] })

In [13]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/twitter4242.txt", "r", encoding="utf-8",errors='ignore').read().split("\n")[1:-1]
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

en_twitter_data = list(data)

In [14]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_dev_complete.txt", encoding="utf-8").read().split("\n")[1:-1]
data += open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_test_average_complete.tsv", encoding="utf-8").read().split("\n")[1:-2]

data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

es2_twitter_data = list(data)

In [15]:
print(len(en_twitter_data))
print(len(es2_twitter_data))
print(len(en_es_wssa_data))

4241
3202
3062


# Load Embedding

In [16]:
import io
import numpy as np

In [17]:
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [66]:
src_path = '../vendors//MUSE/dumped/debug/4u9hakomha/vectors-en.txt'
tgt_path = '../vendors//MUSE/dumped/debug/4u9hakomha/vectors-es.txt'
nmax = 100000  # maximum number of word embeddings to load

src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

# Get nearest neighbors

In [67]:
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [68]:
# printing nearest neighbors in the source space
src_word = 'cat'
get_nn(src_word, src_embeddings, src_id2word, src_embeddings, src_id2word, K=5)

Nearest neighbors of "cat":
1.0000 - cat
0.7322 - cats
0.6453 - kitten
0.6381 - dog
0.6218 - kittens


In [69]:
# printing nearest neighbors in the target space
src_word = 'cat'
get_nn(src_word, src_embeddings, src_id2word, tgt_embeddings, tgt_id2word, K=5)

Nearest neighbors of "cat":
0.6266 - gato
0.5317 - perro
0.5213 - gatito
0.4872 - gorila
0.4767 - ratoncito


In [70]:
src_embeddings.shape

(100000, 300)

# Date Preprocessing 2

In [71]:
MAX_SEQUENCE_LENGTH = 10

In [72]:
from keras.preprocessing import sequence
from keras.utils import to_categorical

In [73]:
embedding_matrix = src_embeddings.copy().tolist()
embedding_matrix.extend(tgt_embeddings.tolist())
embedding_matrix = np.array(embedding_matrix)

In [74]:
id2word = src_id2word.copy()
word2id = src_word2id.copy()
next_id = len(id2word.keys())
counter = len(id2word.keys())
to_be_removed = []
for key in tgt_id2word:
    if tgt_id2word[key] in word2id:
        to_be_removed.append(counter)
        embedding_matrix[word2id[tgt_id2word[key]]] =  (embedding_matrix[word2id[tgt_id2word[key]]] + embedding_matrix[counter])/2
    else:
        id2word[next_id] = tgt_id2word[key]
        word2id[tgt_id2word[key]] = next_id
        next_id += 1
    counter += 1

In [75]:
embedding_matrix = np.delete(embedding_matrix, to_be_removed, axis=0)

In [76]:
print(embedding_matrix.shape)

(161832, 300)


In [77]:
def to_x(sample):
    x = []
    for word in sample['tokens']:
        word = word.lower()
        if word in word2id:
            x.append(word2id[word])
#         else:
#             print(word)
    return x

def to_x_y(data_):
    x=  list(map(lambda x : to_x(x), data_))
    y = list(map(lambda x: x['sentiment'],data_))
    x = list(sequence.pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH))
    y = list(to_categorical(y,num_classes=3))
    return x,y

In [78]:
x_train, y_train =  to_x_y(en_twitter_data)
temp_1, temp_2 = to_x_y(es2_twitter_data)
x_train.extend(temp_1)
y_train.extend(temp_2)
temp_1, temp_2 = to_x_y(es_tass1_data)
x_train.extend(temp_1)
y_train.extend(temp_2)

In [79]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [80]:
print(x_train.shape)
print(y_train.shape)

(14660, 10)
(14660, 3)


In [81]:
get_class_weight(y_train)

{0: 0.9227089627391742, 1: 0.9202762084118016, 2: 1.2053938496957737}

In [82]:
x_test, y_test = to_x_y(en_es_wssa_data)
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

(3062, 10)
(3062, 3)


In [83]:
get_class_weight(y_test)

{0: 0.7773546585427773, 1: 1.0598823122187608, 2: 1.2985581000848176}

# Model

In [84]:
from keras.layers import *
from keras.models import Sequential

In [85]:
#embedding
EMBEDDING_DIM = embedding_matrix.shape[1] 
MAX_NUM_WORDS = embedding_matrix.shape[0]

In [86]:
# Convolution
kernel_size = 5
filters = 64
pool_size = 4

In [87]:
# LSTM
lstm_output_size = 70

In [91]:
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS,
                     EMBEDDING_DIM,
                     weights=[embedding_matrix],
                     input_length=MAX_SEQUENCE_LENGTH,
                     trainable=False))
model.add(Dropout(0.4))
model.add(Conv1D(filters,
                 kernel_size,
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
# model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])

In [92]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 10, 300)           48549600  
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 64)             96064     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 70)                37800     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 213       
_________________________________________________________________
activation_6 (Activation)    (None, 3)                 0         
Total para

In [93]:
history = model.fit(x_train, y_train, epochs=40, batch_size=64, validation_data=(x_test,y_test), shuffle=True)

Train on 14660 samples, validate on 3062 samples
Epoch 1/40
14660/14660 [==============================] - 6s 377us/step - loss: 1.0435 - acc: 0.4521 - f1: 0.1939 - val_loss: 1.0887 - val_acc: 0.3988 - val_f1: 0.2313
Epoch 2/40
14660/14660 [==============================] - 3s 182us/step - loss: 0.9868 - acc: 0.5020 - f1: 0.3819 - val_loss: 1.0557 - val_acc: 0.4288 - val_f1: 0.2204
Epoch 3/40
14660/14660 [==============================] - 3s 212us/step - loss: 0.9555 - acc: 0.5369 - f1: 0.4309 - val_loss: 1.0692 - val_acc: 0.4046 - val_f1: 0.2489
Epoch 4/40
14660/14660 [==============================] - 3s 186us/step - loss: 0.9345 - acc: 0.5473 - f1: 0.4532 - val_loss: 1.0557 - val_acc: 0.4393 - val_f1: 0.2730
Epoch 5/40
14660/14660 [==============================] - 3s 210us/step - loss: 0.9016 - acc: 0.5761 - f1: 0.4969 - val_loss: 1.0612 - val_acc: 0.4370 - val_f1: 0.3129
Epoch 6/40
14660/14660 [==============================] - 3s 182us/step - loss: 0.8701 - acc: 0.5898 - f1: 0.54

In [46]:
import tensorflow as tf

In [47]:
tf.__version__

'1.12.0'

In [32]:
from keras import backend as K


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [33]:
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
def get_class_weight(y):
    """
    Used from: https://stackoverflow.com/a/50695814
    TODO: check validity and 'balanced' option
    :param y: A list of one-hot-encoding labels [[0,0,1,0],[0,0,0,1],..]
    :return: class-weights to be used by keras model.fit(.. class_weight="") -> {0:0.52134, 1:1.adas..}
    """
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    return d_class_weights

In [ ]:
# doesn't work

In [29]:
from dlblocks import text
from dlblocks.pyutils import mapArrays , loadJson , saveJson , selectKeys , oneHotVec , padList
from dlblocks.pyutils import int64Arr , floatArr

In [ ]:
vocab = text.Vocabulary()

for d in es_tass1_data + en_es_wssa_data + en_twitter_data + es2_twitter_data :
    vocab.add_words( d['tokens']  )

    
vocab.keepTopK(25000)



maxSentenceL = 150

def vecc( d ):
    ret = {}
    words   = d['tokens']
    wordids = map( vocab , words )
    ret['sentence'] = int64Arr( padList( wordids , maxSentenceL , 0 , 'left') )
    ret['sentiment_val'] =  floatArr( d['sentiment'] )
    ret['sentiment_id'] =  int64Arr( d['sentiment'] + 1 )
    ret['sentiment_onehot'] =  floatArr( oneHotVec( d['sentiment']+1 , 3  ) )

    return ret





en_es_wssa_data_train_arr = mapArrays( en_es_wssa_data_train , vecc )
en_es_wssa_data_test_arr = mapArrays( en_es_wssa_data_test , vecc )

en_twitter_data_train_arr = mapArrays( en_twitter_data , vecc )
es_tass1_datatrain_arr = mapArrays( es_tass1_data , vecc )

datasets = {"en_es_wssa_data_train_arr":en_es_wssa_data_train_arr ,
           "en_es_wssa_data_test_arr":en_es_wssa_data_test_arr ,
           "en_twitter_data_train_arr":en_twitter_data_train_arr ,
           "es_tass1_datatrain_arr": es_tass1_datatrain_arr }



outFNN = "../data/senti_prepped.h5"

f = h5py.File(outFNN , "w")
for kk in datasets.keys():
    f.create_group( kk  )
    for k in datasets[kk].keys():
        f[ kk ].create_dataset( k , data=datasets[kk][k] )

print "HDF5 file created !"